In [1]:
import re
import json
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import os

# Configuration
MODEL_NAME = "Qwen/Qwen2.5-Math-1.5B-Instruct"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_SAMPLES = 500
CHECKPOINT_INTERVAL = 50
CHECKPOINT_FILE = "gsm8k_baseline_checkpoint.jsonl"
OUTPUT_FILE = "gsm8k_baseline.jsonl"

print(f"Using device: {DEVICE}")
print(f"Loading dataset...")

# Load dataset
dataset = load_dataset("openai/gsm8k", "main", split="test")
dataset = dataset.select(range(min(NUM_SAMPLES, len(dataset))))

print(f"Evaluating on {len(dataset)} samples")
print(f"Loading model: {MODEL_NAME}")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32
).eval()

def normalize_numeric_answer(answer):
    """
    Normalize numeric answers for comparison by:
    - Removing commas (thousands separators)
    - Removing dollar signs and percent signs
    - Removing whitespace
    - Converting to string for comparison
    """
    if not answer:
        return ""

    # Convert to string if not already
    answer = str(answer).strip()

    # Remove common formatting characters
    answer = answer.replace(',', '')
    answer = answer.replace('$', '')
    answer = answer.replace('%', '')
    answer = answer.replace(' ', '')

    # Try to convert to float then back to string to normalize
    # This handles cases like "42.0" vs "42"
    try:
        num = float(answer)

        # Check for infinity or NaN
        if not (-1e15 < num < 1e15):
            return answer

        # If it's a whole number, convert to int
        if num == int(num):
            return str(int(num))
        return str(num)
    except (ValueError, OverflowError):
        return answer

def extract_answer(text):
    """
    Extract answer from model output.
    Priority order:
    1. Content inside \boxed{}
    2. Last number in the text
    """
    # First try to find answer in \boxed{}
    boxed_match = re.search(r"\\boxed\{([^}]+)\}", text)
    if boxed_match:
        return normalize_numeric_answer(boxed_match.group(1))

    # Fallback: extract last number from text
    nums = re.findall(r"-?\d+\.?\d*", text)
    if nums:
        return normalize_numeric_answer(nums[-1])

    return ""

def extract_gold_answer(answer_text):
    """
    Extract the gold answer from GSM8K format.
    GSM8K answers end with #### followed by the final answer.
    """
    # GSM8K format: solution text followed by #### answer
    parts = answer_text.strip().split("####")
    if len(parts) >= 2:
        gold = parts[-1].strip()
    else:
        # Fallback: take last number
        nums = re.findall(r"-?\d+\.?\d*", answer_text)
        gold = nums[-1] if nums else ""

    return normalize_numeric_answer(gold)

# Check if checkpoint exists and load progress
start_idx = 0
results = []
correct = 0

if os.path.exists(CHECKPOINT_FILE):
    print(f"Loading checkpoint from {CHECKPOINT_FILE}")
    with open(CHECKPOINT_FILE, "r", encoding="utf-8") as f:
        for line in f:
            result = json.loads(line)
            results.append(result)
            if result["correct"]:
                correct += 1
    start_idx = len(results)
    print(f"Resuming from sample {start_idx}")

# Evaluation loop
print("Starting evaluation...")
for idx in tqdm(range(start_idx, len(dataset)), initial=start_idx, total=len(dataset)):
    ex = dataset[idx]
    prompt = ex['question']

    # Prepare messages
    messages = [
        {"role": "system", "content": "Please reason step by step, and put your final answer within \\boxed{}."},
        {"role": "user", "content": prompt}
    ]

    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Tokenize and generate
    inputs = tokenizer([text], return_tensors="pt").to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode only the generated tokens
    generated_ids = [
        out[len(inp):] for inp, out in zip(inputs.input_ids, output_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Extract answers
    pred_ans = extract_answer(response)
    gold_ans = extract_gold_answer(ex["answer"])

    # Compare
    is_correct = (pred_ans == gold_ans) and (pred_ans != "")
    if is_correct:
        correct += 1

    # Print progress
    status = "✓" if is_correct else "✗"
    print(f"\n{status} [{idx+1}/{len(dataset)}] Gold: {gold_ans} | Pred: {pred_ans}")

    # Store result
    results.append({
        "question": prompt,
        "gold": gold_ans,
        "predicted": pred_ans,
        "model_output": response,
        "correct": is_correct
    })

    # Save checkpoint periodically
    if (idx + 1) % CHECKPOINT_INTERVAL == 0:
        with open(CHECKPOINT_FILE, "w", encoding="utf-8") as f:
            for r in results:
                json.dump(r, f, ensure_ascii=False)
                f.write("\n")
        print(f"Checkpoint saved at sample {idx+1}")

# Save final results
print(f"\nSaving final results to {OUTPUT_FILE}")
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    for r in results:
        json.dump(r, f, ensure_ascii=False)
        f.write("\n")

# Calculate and display final accuracy
accuracy = correct / len(dataset)
print(f"\n{'='*60}")
print(f"GSM8K BASELINE EVALUATION COMPLETE")
print(f"{'='*60}")
print(f"Model: {MODEL_NAME}")
print(f"Samples evaluated: {len(dataset)}")
print(f"Correct: {correct}")
print(f"Incorrect: {len(dataset) - correct}")
print(f"Accuracy: {accuracy:.2%} ({correct}/{len(dataset)})")
print(f"{'='*60}")

# Save summary
summary = {
    "model": MODEL_NAME,
    "total_samples": len(dataset),
    "correct": correct,
    "incorrect": len(dataset) - correct,
    "accuracy": accuracy
}

with open("gsm8k_summary.json", "w", encoding="utf-8") as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)

print(f"Summary saved to gsm8k_summary.json")

Using device: cuda
Loading dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

...


✓ [498/500] Gold: 112 | Pred: 112


100%|█████████▉| 499/500 [1:43:35<00:10, 10.52s/it]


✓ [499/500] Gold: 40 | Pred: 40


100%|██████████| 500/500 [1:43:47<00:00, 12.46s/it]


✗ [500/500] Gold: 10 | Pred: 8
Checkpoint saved at sample 500

Saving final results to gsm8k_baseline.jsonl

GSM8K BASELINE EVALUATION COMPLETE
Model: Qwen/Qwen2.5-Math-1.5B-Instruct
Samples evaluated: 500
Correct: 425
Incorrect: 75
Accuracy: 85.00% (425/500)
Summary saved to gsm8k_summary.json
